In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.constraints import maxnorm
from keras.optimizers import adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.
/Users/jackholder/anaconda2/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
np.random.seed(10)

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [4]:
import tensorflow as tf
from sklearn.utils.random import sample_without_replacement
import scipy.sparse as sp
from sklearn.utils import check_random_state


def gaussian_random_tensor(n_components, n_features, random_state=None):
    rng = check_random_state(random_state)
    components = rng.normal(loc=0.0,
                            scale=1.0 / np.sqrt(n_components),
                            size=(n_components, n_features))
    components = components.T
    # components = np.sign(components)
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def _check_density(density, n_features):
    """Factorize density check according to Li et al."""
    if density == 'auto':
        density = 1 / np.sqrt(n_features)
    elif density <= 0 or density > 1:
        raise ValueError("Expected density in range ]0, 1], got: %r"
                         % density)
    return density

def sparse_random_tensor(n_components, n_features, density='auto',
                         random_state=None):
 
    density = _check_density(density, n_features)
    rng = check_random_state(random_state)
    if density == 1:
        # skip index generation if totally dense
        components = rng.binomial(1, 0.5, (n_components, n_features)) * 2 - 1
        return 1 / np.sqrt(n_components) * components
    else:
        indices = []
        offset = 0
        indptr = [offset]
        for i in range(n_components):
            # find the indices of the non-zero components for row i
            n_nonzero_i = rng.binomial(n_features, density)
            indices_i = sample_without_replacement(n_features, n_nonzero_i,
                                                   random_state=rng)
            indices.append(indices_i)
            offset += n_nonzero_i
            indptr.append(offset)
        indices = np.concatenate(indices)
        # Among non zero components the probability of the sign is 50%/50%
        data = rng.binomial(1, 0.5, size=np.size(indices)) * 2 - 1
        # build the CSR structure by concatenating the rows
        components = sp.csr_matrix((data, indices, indptr),
                                   shape=(n_components, n_features))
        return tf.convert_to_tensor(np.sqrt(1 / density) / np.sqrt(n_components) * components)


def project(x, ncomp): #ncomp is the number of dimensions we want to shrink to 
    features = K.int_shape(x)[1]
    Y = gaussian_random_tensor(ncomp, features)
    X_new = K.dot(x, Y)
    return X_new

In [10]:
dims = [4608, 3456, 2304, 1152, 576, 288, 144]
import time
accuracy = []
size = []
times = []
epochs = 10
for d in dims:
    start = time.time()
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(1,28,28), kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    if d != 4608:
        model.add(Lambda (lambda x: project(x, d)))
    model.add(Dense(d, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    h = model.fit(X_train, Y_train, batch_size=32, epochs=epochs, verbose=1)
    
    a = model.evaluate(X_test, Y_test, verbose=0)[1]
    
    end = time.time()
    t = end - start
    accuracy.append(a)
    size.append(d)
    times.append(t)

import pandas as pd
df = pd.DataFrame({'Size': size, 'Accuracy': accuracy, 'Runtime': times})


Epoch 1/10
60000/60000 [==============================] - 518s - loss: 0.1197 - acc: 0.9625   
Epoch 2/10
60000/60000 [==============================] - 500s - loss: 0.0455 - acc: 0.9862   
Epoch 3/10
60000/60000 [==============================] - 499s - loss: 0.0314 - acc: 0.9903   
Epoch 4/10
60000/60000 [==============================] - 497s - loss: 0.0239 - acc: 0.9928   
Epoch 5/10
60000/60000 [==============================] - 497s - loss: 0.0213 - acc: 0.9931   
Epoch 6/10
60000/60000 [==============================] - 504s - loss: 0.0156 - acc: 0.9951   
Epoch 7/10
60000/60000 [==============================] - 500s - loss: 0.0155 - acc: 0.9952   
Epoch 8/10
60000/60000 [==============================] - 498s - loss: 0.0126 - acc: 0.9963   
Epoch 9/10
60000/60000 [==============================] - 497s - loss: 0.0115 - acc: 0.9967   
Epoch 10/10
60000/60000 [==============================] - 497s - loss: 0.0123 - acc: 0.9963   
Epoch 1/10
60000/60000 [=========================

In [11]:
df['time_saved'] = (max(df['Runtime']) - df['Runtime']) / max(df['Runtime'])
df['acc_lost'] = (max(df['Accuracy']) - df['Accuracy'])
df.head()

,Accuracy,Runtime,Size,time_saved,acc_lost
0,0.9891,5043.912917,4608,0.260923,0.0019
1,0.9910,6824.607547,3456,0.000000,0.0000
2,0.9867,4590.439886,2304,0.327369,0.0043
3,0.9884,3224.516772,1152,0.527516,0.0026
4,0.9893,2847.549422,576,0.582753,0.0017


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_style("whitegrid")
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')